# Import Libraries

In [1]:
import time
from qwikidata.sparql  import return_sparql_query_results
from qwikidata.linked_data_interface import get_entity_dict_from_api
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import urllib.request
import json
import string
from difflib import SequenceMatcher
from viapy.api import ViafAPI
import re
import math
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
location_to_save = "data/final_files/"
large_location_to_save = "data/final_files/large_files/"

# Let's see.

In [10]:
def remove_redundant_books(author_df):
    author_df.alt_title = author_df.alt_title.apply(lambda x: simplify(x))
    unique_titles = author_df.alt_title.unique()
    author_df_alt = author_df.copy()
    for unique_title in unique_titles:
        first_ISBN = author_df.loc[author_df.alt_title == unique_title].iloc[0].ISBN
        author_df_alt.loc[author_df_alt.alt_title == unique_title, "ISBN_alt"] = first_ISBN
        ISBN_dict = dict(zip(author_df_alt.ISBN, author_df_alt.ISBN_alt))
    return(ISBN_dict)

In [11]:
def simplify(name):
    name = name.replace(" ","").translate(str.maketrans('', '', string.punctuation)).lower()
    return name

In [12]:
entire_books = pd.read_csv(location_to_save+"items_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")

In [13]:
entire_books["ISBN_alt"] = ""

In [14]:
unique_authors = entire_books[["QID", "viaf_id"]].drop_duplicates()

In [15]:
len(unique_authors)

82180

In [16]:
start = time.time()
large_ISBN_dict = {}
for index, row in unique_authors.iterrows():
    viaf_id = row.viaf_id
    QID = row.QID
    author_df = entire_books[(entire_books.QID==QID)&(entire_books.viaf_id==viaf_id)][["ISBN", 
                                                                                       "title",
                                                                                      "alt_title",
                                                                                      "ISBN_alt"]]
    ISBN_dict = remove_redundant_books(author_df)
    large_ISBN_dict.update(ISBN_dict)
    del author_df
    del ISBN_dict
    if index%500==0:
        print("500 is gone!!!!!!!!!", index)
        print(time.time()-start)
        start = time.time()
        with open('ISBN_dict.pkl', 'wb') as handle:
            pkl.dump(large_ISBN_dict, handle)


500 is gone!!!!!!!!! 0
0.03922414779663086
500 is gone!!!!!!!!! 500
44.35216474533081
500 is gone!!!!!!!!! 1000
18.96273422241211
500 is gone!!!!!!!!! 2500
45.39629101753235
500 is gone!!!!!!!!! 3500
26.397749423980713
500 is gone!!!!!!!!! 5500
46.487560749053955
500 is gone!!!!!!!!! 7000
29.872409343719482
500 is gone!!!!!!!!! 8500
25.93448281288147
500 is gone!!!!!!!!! 10500
38.06102895736694
500 is gone!!!!!!!!! 11000
9.25963807106018
500 is gone!!!!!!!!! 12500
27.561559677124023
500 is gone!!!!!!!!! 13000
8.866279363632202
500 is gone!!!!!!!!! 14500
25.817970514297485
500 is gone!!!!!!!!! 16000
25.585728645324707
500 is gone!!!!!!!!! 17000
17.415254831314087
500 is gone!!!!!!!!! 17500
9.301963806152344
500 is gone!!!!!!!!! 19000
24.592336893081665
500 is gone!!!!!!!!! 20500
21.878334760665894
500 is gone!!!!!!!!! 21000
5.982553005218506
500 is gone!!!!!!!!! 21500
6.187631845474243
500 is gone!!!!!!!!! 22000
6.879296779632568
500 is gone!!!!!!!!! 23000
11.767699956893921
500 is gone

In [17]:
 with open('ISBN_dict.pkl', 'wb') as handle:
            pkl.dump(large_ISBN_dict, handle)

In [18]:
len(large_ISBN_dict.keys())

271360

In [19]:
len(np.unique(list(large_ISBN_dict.keys())))

271360

In [20]:
len(np.unique(list(large_ISBN_dict.values())))

220266

# Result

In [3]:
with open('ISBN_dict.pkl', 'rb') as handle:
        large_ISBN_dict = pkl.load(handle)

# Fill in

In [13]:
entire_books["ISBN_alt"] = ""

In [14]:
for index, row in entire_books.iterrows():
    if row.alt_title!="":
        ISBN = row.ISBN
        ISBN_alt = large_ISBN_dict[ISBN]
        entire_books.at[index, "ISBN_alt"] = ISBN_alt

In [15]:
entire_books.to_csv(location_to_save+"items_books_with_readable_properties_correct_ISBN.csv")